In [14]:
# Import necessary libraries and initialize a SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [15]:
# Initialize a Spark session
spark = SparkSession.builder.appName("TopProductCategoriesRevenue").getOrCreate()

23/09/12 07:57:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [27]:
# Load the necessary datasets
products_df = spark.read.csv("Data/olist_products_dataset.csv", header=True, inferSchema=True)
order_items_df = spark.read.csv("Data/olist_order_items_dataset.csv", header=True, inferSchema=True)

In [28]:

# Load the necessary dataset
payments_df = spark.read.csv("Data/olist_order_payments_dataset.csv", header=True, inferSchema=True)

In [41]:

from pyspark.sql.functions import avg, col



# Load the required datasets into DataFrames
seller_df = spark.read.csv("Data/olist_sellers_dataset.csv", header=True, inferSchema=True)
order_items_df = spark.read.csv("Data/olist_order_items_dataset.csv", header=True, inferSchema=True)
orders_df = spark.read.csv("Data/olist_order_items_dataset.csv", header=True, inferSchema=True)
reviews_df = spark.read.csv("Data/olist_order_reviews_dataset.csv", header=True, inferSchema=True)
# Calculate the average delivery time for each seller
delivery_time_df = (
    order_items_df
    .join(seller_df, order_items_df["Seller_id"] == seller_df["Seller_id"])
    .join(orders_df, order_items_df["Order_id"] == orders_df["Order_id"])
    .withColumn("Delivery_Time", (col("Order_delivered_customer_date").cast("timestamp").cast("long") - col("Order_delivered_carrier_data").cast("timestamp").cast("long")) / (3600 * 24))  # Calculate delivery time in days
    .groupBy(seller_df["Seller_id"])
    .agg(avg("Delivery_Time").alias("Avg_Delivery_Time"))
    .orderBy("Avg_Delivery_Time", ascending=False)
)

# Identify sellers with the highest and lowest customer ratings
seller_ratings_df = (
    seller_df
    .join(order_items_df, seller_df["Seller_id"] == order_items_df["Seller_id"])
    .join(orders_df, order_items_df["Order_id"] == orders_df["Order_id"])
    .join(reviews_df, orders_df["Order_id"] == reviews_df["order_id"], "left")
    .groupBy(seller_df["Seller_id"])
    .agg(avg(col("review_score").cast("float")).alias("Avg_Review_Score"))
    .orderBy("Avg_Review_Score", ascending=False)
)

# Show the results
delivery_time_df.show()
seller_ratings_df.show()
# # Calculate the average delivery time for each seller
# delivery_time_df = (
#     order_items_df
#     .join(seller_df, order_items_df.Seller_id == seller_df.Seller_id)
#     .join(orders_df, order_items_df.Order_id == orders_df.Order_id)
#     .withColumn("Delivery_Time", (col("Order_delivered_customer_date").cast("timestamp").cast("long") - col("Order_delivered_carrier_data").cast("timestamp").cast("long")) / (3600 * 24))  # Calculate delivery time in days
#     .groupBy("seller_df.Seller_id")
#     .agg(avg("Delivery_Time").alias("Avg_Delivery_Time"))
#     .orderBy("Avg_Delivery_Time", ascending=False)
# )

# # Identify sellers with the highest and lowest customer ratings
# seller_ratings_df = (
#     seller_df
#     .join(order_items_df, seller_df.Seller_id == order_items_df.Seller_id)
#     .join(orders_df, order_items_df.Order_id == orders_df.Order_id)
#     .join(reviews_df, orders_df.Order_id == reviews_df.order_id, "left")
#     .groupBy("seller_df.Seller_id")
#     .agg(avg(col("review_score").cast("float")).alias("Avg_Review_Score"))
#     .orderBy("Avg_Review_Score", ascending=False)
# )

# # Show the results
# delivery_time_df.show()
# seller_ratings_df.show()

# Stop the Spark session
spark.stop()


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Order_delivered_customer_date` cannot be resolved. Did you mean one of the following? [`seller_state`, `order_item_id`, `order_item_id`, `shipping_limit_date`, `shipping_limit_date`].;
'Project [order_id#2574, order_item_id#2575, product_id#2576, seller_id#2577, shipping_limit_date#2578, price#2579, freight_value#2580, seller_id#2549, seller_zip_code_prefix#2550, seller_city#2551, seller_state#2552, order_id#2605, order_item_id#2606, product_id#2607, seller_id#2608, shipping_limit_date#2609, price#2610, freight_value#2611, ((cast(cast('Order_delivered_customer_date as timestamp) as bigint) - cast(cast('Order_delivered_carrier_data as timestamp) as bigint)) / 86400) AS Delivery_Time#2711]
+- Join Inner, (Order_id#2574 = Order_id#2605)
   :- Join Inner, (Seller_id#2577 = Seller_id#2549)
   :  :- Relation [order_id#2574,order_item_id#2575,product_id#2576,seller_id#2577,shipping_limit_date#2578

In [39]:
from pyspark.sql.functions import col, count, avg, stddev
window_spec = Window.partitionBy("Order_id")



# Alias the payments_df as "payments" for clarity
payments_df = payments_df.alias("payments")

# Define a window specification to partition by order_id
window_spec = Window.partitionBy("payments.Order_id")

# Calculate the count of payment methods per order
payment_count_per_order = payments_df.withColumn("payment_method_count", count("payments.Payment_type").over(window_spec))
payment_count_per_order  = payment_count_per_order .withColumnRenamed("Payment_type", "new_payment_types")
# Calculate the average and standard deviation of payment installments per order
payment_stats_per_order = payments_df.withColumn("avg_installments", avg("payments.Payment_installments").over(window_spec)) \
                                     .withColumn("stddev_installments", stddev("payments.Payment_installments").over(window_spec))
payment_stats_per_order = payment_stats_per_order.withColumnRenamed("Payment_type", "new_payment_type")
# Define thresholds for detecting unusual payment patterns
max_payment_methods = 1  # Maximum allowed payment methods per order
max_avg_installments = 3  # Maximum allowed average installments per order
max_stddev_installments = 2  # Maximum allowed standard deviation of installments per order

# Detect potential fraud by filtering orders with unusual payment patterns
potential_fraud_orders = payments_df.join(payment_count_per_order, "Order_id", "inner") \
                                    .join(payment_stats_per_order, "Order_id", "inner") \
                                    .filter((col("payment_method_count") > max_payment_methods) |
                                            (col("avg_installments") > max_avg_installments) |
                                            (col("stddev_installments") > max_stddev_installments)) 
# potential_fraud_orders.select("payments.Order_id", "payments.Payment_type".alias("payment_type"),
#                                             "payments.Payment_installments",
#                                             col("payment_method_count"), col("avg_installments"), col("stddev_installments"))

selected_columns = potential_fraud_orders.select(
    payments_df["Order_id"],
    payments_df[col("Payment_type")],
    payments_df[col("Payment_installments")],
    col("payment_method_count"),
    col("avg_installments"),
    col("stddev_installments")
)

# Show potential fraud orders
selected_columns.show()
# # Show potential fraud orders
# potential_fraud_orders.show()

AnalysisException: [DATATYPE_MISMATCH.FILTER_NOT_BOOLEAN] Cannot resolve "Payment_type" due to data type mismatch: Filter expression "Payment_type" of type "STRING" is not a boolean.;
Filter Payment_type#1357: string
+- SubqueryAlias payments
   +- SubqueryAlias payments
      +- SubqueryAlias payments
         +- SubqueryAlias payments
            +- SubqueryAlias payments
               +- SubqueryAlias payments
                  +- SubqueryAlias payments
                     +- SubqueryAlias payments
                        +- Relation [order_id#1355,payment_sequential#1356,payment_type#1357,payment_installments#1358,payment_value#1359] csv


In [5]:
# Join the product and order items datasets to get product revenue
joined_df = products_df.join(order_items_df, "product_id", "inner")


# Join the product and order items datasets to get product revenue
joined_df = products_df.join(order_items_df, "product_id", "inner")

In [6]:
# Calculate the revenue for each product by multiplying price and quantity
revenue_df = joined_df.withColumn("total_revenue", col("price") * col("order_item_id"))

In [7]:
# Group by product category name and calculate the total revenue for each category
category_revenue_df = revenue_df.groupBy("product_category_name") \
                                .agg(sum("total_revenue").alias("category_total_revenue"))

In [8]:
# Rank the categories by total revenue in descending order
window_spec = Window.orderBy(col("category_total_revenue").desc())
ranked_category_revenue_df = category_revenue_df.withColumn("rank", F.rank().over(window_spec))

In [9]:
# Select the top 5 product categories
top_5_categories = ranked_category_revenue_df.filter(col("rank") <= 5).select("product_category_name", "category_total_revenue")

# Show the result
top_5_categories.show()


23/09/12 07:34:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 07:34:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 07:34:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 07:34:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 07:34:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 07:34:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 0

+---------------------+----------------------+
|product_category_name|category_total_revenue|
+---------------------+----------------------+
|         beleza_saude|    1347468.4900000044|
|   relogios_presentes|    1259634.5800000038|
|      cama_mesa_banho|    1228795.4600000102|
| informatica_acess...|    1135454.6400000013|
|        esporte_lazer|     1082435.419999999|
+---------------------+----------------------+



In [11]:
# Load the necessary dataset
payments_df = spark.read.csv("Data/olist_order_payments_dataset.csv", header=True, inferSchema=True)
orders_df = spark.read.csv("Data/olist_orders_dataset.csv", header=True, inferSchema=True)

# Convert the timestamp columns to date format

orders_df = orders_df.withColumn("Order_purchase_month", F.date_format(col("Order_purchase_timestamp"), "yyyy-MM"))

# Join the payments and orders datasets on the order_id
joined_df = payments_df.join(orders_df, "Order_id", "inner")

# Group by payment type, purchase month, and count the number of orders and average payment installments
payment_analysis_df = joined_df.groupBy("Payment_type", "Order_purchase_month") \
    .agg(
        F.countDistinct("Order_id").alias("num_orders"),
        F.avg("Payment_installments").alias("avg_installments")
    ) \
    .orderBy("Order_purchase_month")

# Show the result
payment_analysis_df.show()



+------------+--------------------+----------+------------------+
|Payment_type|Order_purchase_month|num_orders|  avg_installments|
+------------+--------------------+----------+------------------+
| credit_card|             2016-09|         3|               2.0|
|      boleto|             2016-10|        63|               1.0|
| credit_card|             2016-10|       253| 4.405511811023622|
|     voucher|             2016-10|        11|               1.0|
|  debit_card|             2016-10|         2|               1.0|
| credit_card|             2016-12|         1|               1.0|
|      boleto|             2017-01|       197|               1.0|
|  debit_card|             2017-01|         9|               1.0|
| credit_card|             2017-01|       582| 3.795883361921098|
|     voucher|             2017-01|        33|               1.0|
|     voucher|             2017-02|        69|               1.0|
|  debit_card|             2017-02|        13|               1.0|
| credit_c

In [1]:

import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql import WindowSpec



# Convert the timestamp columns to date format

orders_df = orders_df.withColumn("Order_purchase_month", F.date_format(col("Order_purchase_timestamp"), "yyyy-MM"))

# Join the payments and orders datasets on the order_id
joined_df = payments_df.join(orders_df, "Order_id", "inner")

# Group by payment type, purchase month, and calculate various statistics on payment installments
window_spec = Window.partitionBy("Payment_type", "Order_purchase_month")

payment_analysis_df = joined_df.groupBy("Payment_type", "Order_purchase_month") \
    .agg(
        F.countDistinct("Order_id").alias("num_orders"),
        F.avg("Payment_installments").alias("avg_installments"),
        F.stddev("Payment_installments").alias("stddev_installments"),
        F.expr("percentile_approx(Payment_installments, 0.5)").alias("median_installments")
    ) \
    .orderBy("Order_purchase_month")

# Show the result
payment_analysis_df.show()

# Stop the Spark session
spark.stop()


NameError: name 'orders_df' is not defined